In [ ]:
%load_ext autoreload 
#allows you to reload this module without having to restart kernel
%aimport rugpeek_functions 
import matplotlib.pyplot as plt
import numpy as np
import lmfit as lf
import matplotlib as mpl
import sys

Define some data filenames to read in

In [ ]:
data_file_ferric_soret_shg0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_1_matrix"
data_file_ferric_soret_shg1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_2_matrix"
data_file_ferric_soret_shg2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_SHG_10kHz_TA_3_matrix"

data_file_ferric_soret_fun0 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_1_matrix"
data_file_ferric_soret_fun1 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_2_matrix"
data_file_ferric_soret_fun2 = "../data/aarhus_TA_jan24/2024-01-12/FerricMb_80uM_409nm_FUN_10kHz_TA_3_matrix"

solvent_ferric_soret_shg = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_SHG_chirp_1_matrix"
solvent_ferric_soret_fun = "../data/aarhus_TA_jan24/2024-01-16/milliQ_409nm_FUN_chirp_1_matrix"

Load them as instances of the Rug class - here wrapped within an average function in case you want to average more than one (commented out in example)

In [ ]:
%autoreload 1
rp = rugpeek_functions
#JDP create an instance of the Rug class
ferric_soret_shg = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_shg0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_shg1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_shg2, ".dat")])

ferric_soret_fun = rp.RugTools.average_rugs([rp.Rug(data_file_ferric_soret_fun0, ".dat")])
                                            # rp.Rug(data_file_ferric_soret_fun1, ".dat"),
                                            # rp.Rug(data_file_ferric_soret_fun2, ".dat")])

solvent_soret_shg = rp.Rug(solvent_ferric_soret_shg, ".dat")
solvent_soret_fun = rp.Rug(solvent_ferric_soret_fun, ".dat")


Apply dispersion corrections. Get them from the solvent only file and then apply the coefficients to the actual data.

In [ ]:
%matplotlib qt5

solvent_soret_shg.get_dispersion_correction_eye(degree=2)
coefs_soret_shg = solvent_soret_shg.dispersion_coefs
solvent_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.apply_dispersion_correction(coefs=coefs_soret_shg)
ferric_soret_shg.offset(coef=coefs_soret_shg[-1])


solvent_soret_fun.get_dispersion_correction_eye(degree=2)
coefs_soret_fun = solvent_soret_fun.dispersion_coefs
ferric_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)
solvent_soret_fun.apply_dispersion_correction(coefs=coefs_soret_fun)
ferric_soret_fun.offset(coef=coefs_soret_fun[-1])

plt.close('all')


Deep copy the corrected rugs so that we can easily get back to them if we break something later. Then stitch together the SHG and FUN regions into one big dataset.

In [ ]:
import copy
ferric_soret_shg_mod = copy.deepcopy(ferric_soret_shg)
ferric_soret_fun_mod = copy.deepcopy(ferric_soret_fun)
solvent_soret_shg_mod = copy.deepcopy(solvent_soret_shg)
solvent_soret_fun_mod = copy.deepcopy(solvent_soret_fun)

ferric_soret_rugs = [ferric_soret_shg_mod, ferric_soret_fun_mod]
ferric_soret_total = rp.RugTools.combine_rugs_wavelengths(ferric_soret_rugs, "ferric_soret_combined_matrix")

solvent_soret_rugs = [solvent_soret_shg_mod, solvent_soret_fun_mod]
solvent_soret_total = rp.RugTools.combine_rugs_wavelengths(solvent_soret_rugs, "solvent_soret_combined_matrix")


Cut out wavelengths that are contaminated by scattering/pump light etc. Best to background subtract after you've done this.

In [ ]:
%autoreload 1
ferric_soret_total.cut_wavelengths([(395,430), (512,518)], fill=0)

ferric_soret_total.background_subtract()


#JDP uncomment if you want to undo operations on the rug.
#ferric_soret_total.reset_matrix()

Compute and plot an SVD - see functions for parameters.

In [ ]:
%autoreload 1
plt.close('all')

#JDP uncomment if you want to limit stuff
#ferric_soret_total.limit_times(tmin=10, tmax=1000)
#ferric_soret_total.limit_wavelengths(wlmin=413, wlmax=700)
                                                       
ferric_soret_total.compute_SVD(threshold=100)

ferric_soret_total.plot_SVD()


Small helper functions that let you get traces and spectra at given times/wavelengths - example below.

In [ ]:

wavelengths = [460, 455, 450, 445, 440, 435, 390, 385, 380, 375, 370]
decays = ferric_soret_total.get_time_trace(wavelengths, plot=True)

times = [-1, 0, 1, 10, 100]
spectra = ferric_soret_total.get_wavelength_trace(times, plot=True)



Do some fits using the global fitter - slightly WIP....

In [ ]:
#  paramdata needs to include all the parameters that are used to fit one dataset
# if using inbuilt lmfit models the names need to match what is expected 
# numbers are just used to differentiate them, and are replaced with prefixes later 


# need to fix this so it checks the length of the params dictionary
paramdata = {'center': [0, -0.3, 0.3, True],
             'sigma': [0.4, 0.0, 0.5, True],
             'gamma': [1, 1E-6, 20, True],
             'amplitude': [0, -5, 5, True],

             'center2': [0, -0.3, 0.3, True],
             'sigma2': [0.4, 0.0, 0.5, True],
             'gamma2': [1, 1E-6, 20, True],
             'amplitude2': [0, -5, 5, True],

             'center3': [0, -0.3, 0.3, True],
             'sigma3': [0.4, 0.0, 0.5, True],
             'gamma3': [1, 1E-6, 30, True],
             'amplitude3': [0, -5, 5, True]}
           #  'c': [0, 0, 2, False]
           # }


#need a way to check that the number of suppplied params matches expected components

#these are bits that are common to each component and to each dataset
fixed_vars = ['center','sigma']

#these are bits that apply to one whole dataset, but vary across different datasets ('offsets')
offset_vars = [] 

#these are bits that vary across components, but are commmon to each dataset
fixed_data_vars = ['gamma'] 

# wavelengths of time slices to fit to
wavelengths = [455, 450, 445, 440, 435, 390, 385, 380]
traces = np.array(ferric_soret_total.get_time_trace(wavelengths))
t = ferric_soret_total.times
ncpts = 2
vars_per_cpt = 4

#call the global fitter to fit to a sum of exponential gaussians
out = rp.RugFits.global_fitter(t, traces, paramdata, None, None, ncpts=ncpts, vars_per_cpt=vars_per_cpt, 
                    prefixdata='d', prefixcpt='c', model=lf.models.ExponentialGaussianModel,
                    method='leastsq', fixed_vars=fixed_vars, offset_vars=offset_vars, fixed_data_vars=fixed_data_vars)

#plot the report. make print true for details
rp.RugFits.plot_fit_result(out, t, traces, wavelengths, ncpts, print_report=False)